# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
##from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-azure-ml'

experiment=Experiment(ws, experiment_name)
project_folder = './pipeline-project'

In [27]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().




Found existing cluster, use it.
Succeeded.............................................................................................

In [28]:



#Dataset

found = False
key = "Capstonedataset"
description_text = "Heart DataSet for Udacity Capstone"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("found!")

if not found:
        # Create AML Dataset and register it into Workspace
        print("not found!")
        example_data = 'heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
##df.describe()




found!


In [29]:
import pandas as pd
import numpy as np

##df = pd.read_csv('C:/Dataset.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]

train.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,split
count,23089.000000,23089.000000,23089.000000,23089.000000,23089.000000,23089.000000,23089.000000,23089.000000,23089.000000,23089.000000,23089.000000
mean,39.983888,256.111395,2.555199,962.225865,0.177141,0.067967,93.570787,-40.530733,3.607024,5166.614362,0.001808
std,10.441725,257.660314,2.772641,187.534465,0.502053,1.573499,0.579821,4.636774,1.737578,72.205590,1.001831
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000,-3.748894
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000,-0.671155
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.444000,-41.800000,4.857000,5191.000000,-0.000235
75%,47.000000,317.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000,0.677735
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000,4.081935


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [34]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target = compute_target,
    training_data=train,
    label_column_name='y',
    n_cross_validations = 3,
    primary_metric = 'accuracy',
    enable_early_stopping = True,
)

print("done.")

done.


In [35]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

ConfigException: ConfigException:
	Message: Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service